In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# Load the first 2000 rows of the dataset
df = pd.read_csv('/content/drive/MyDrive/Adobe/TrainSet.csv', nrows=100)

# Display the first few rows and the columns to confirm loading
display(df.head())
display(df.columns)

,id,date,likes,content,username,media,inferred company
0,1,2020-12-12 00:47:00,1,"Spend your weekend morning with a Ham, Egg, an...",TimHortonsPH,[Photo(previewUrl='https://pbs.twimg.com/media...,tim hortons
1,2,2018-06-30 10:04:20,2750,Watch rapper <mention> freestyle for over an H...,IndyMusic,[Photo(previewUrl='https://pbs.twimg.com/media...,independent
2,3,2020-09-29 19:47:28,57,Canadian Armenian community demands ban on mil...,CBCCanada,[Photo(previewUrl='https://pbs.twimg.com/media...,cbc
3,4,2020-10-01 11:40:09,152,"1st in Europe to be devastated by COVID-19, It...",MKWilliamsRome,[Photo(previewUrl='https://pbs.twimg.com/media...,williams
4,5,2018-10-19 14:30:46,41,Congratulations to Pauletha Butts of <mention>...,BGISD,[Photo(previewUrl='https://pbs.twimg.com/media...,independent


Index(['id', 'date', 'likes', 'content', 'username', 'media',
       'inferred company'],
      dtype='object')

In [ ]:
import re

def extract_image_url(media_list):
    """Extracts the preview or thumbnail URL from the media list using regex."""
    if isinstance(media_list, str):
        # Try to find previewUrl
        preview_match = re.search(r"previewUrl='(https?://[^']+)'", media_list)
        if preview_match:
            extracted_url = preview_match.group(1)
            # Print the extracted URL for debugging
            print(f"Extracted previewUrl (regex): {extracted_url}")
            return extracted_url

        # If previewUrl not found, try to find thumbnailUrl
        thumbnail_match = re.search(r"thumbnailUrl='(https?://[^']+)'", media_list)
        if thumbnail_match:
            extracted_url = thumbnail_match.group(1)
            # Print the extracted URL for debugging
            print(f"Extracted thumbnailUrl (regex): {extracted_url}")
            return extracted_url

    # Print a message if no URL is found
    print(f"No URL found in media list: {media_list}")
    return None

# Apply the function to the 'media' column to create a new column with URLs
df['image_url'] = df['media'].apply(extract_image_url)

# Display the first few rows with the new column
display(df[['media', 'image_url']].head())

Extracted previewUrl (regex): https://pbs.twimg.com/media/Eo8N3JLVoAAlDJT?format=jpg&name=small
Extracted previewUrl (regex): https://pbs.twimg.com/media/Dg7mW-VX0AE_hMn?format=jpg&name=small
Extracted previewUrl (regex): https://pbs.twimg.com/media/EjG2s4aXgAMNM1o?format=jpg&name=small
Extracted previewUrl (regex): https://pbs.twimg.com/media/EjPaVniX0AAaWLJ?format=jpg&name=small
Extracted previewUrl (regex): https://pbs.twimg.com/media/Dp4L0cSUcAAh9JG?format=jpg&name=small
Extracted thumbnailUrl (regex): https://pbs.twimg.com/amplify_video_thumb/1322382278794858496/img/eKRa2rXniZ_eOCNB.jpg
Extracted previewUrl (regex): https://pbs.twimg.com/media/EHo1yZyU0AIBW8N?format=jpg&name=small
Extracted thumbnailUrl (regex): https://pbs.twimg.com/amplify_video_thumb/1019341984543268866/img/ieqbfTc82OlF0hD6.jpg
Extracted previewUrl (regex): https://pbs.twimg.com/media/D2qiPBTX0AI_13F?format=jpg&name=small
Extracted previewUrl (regex): https://pbs.twimg.com/media/EeT6pQZVoAAM2Lj?format=jpg&name=

,media,image_url
0,[Photo(previewUrl='https://pbs.twimg.com/media...,https://pbs.twimg.com/media/Eo8N3JLVoAAlDJT?fo...
1,[Photo(previewUrl='https://pbs.twimg.com/media...,https://pbs.twimg.com/media/Dg7mW-VX0AE_hMn?fo...
2,[Photo(previewUrl='https://pbs.twimg.com/media...,https://pbs.twimg.com/media/EjG2s4aXgAMNM1o?fo...
3,[Photo(previewUrl='https://pbs.twimg.com/media...,https://pbs.twimg.com/media/EjPaVniX0AAaWLJ?fo...
4,[Photo(previewUrl='https://pbs.twimg.com/media...,https://pbs.twimg.com/media/Dp4L0cSUcAAh9JG?fo...


In [ ]:
import requests
from requests.exceptions import RequestException

def is_url_valid(url):
    """Checks if a URL is valid and accessible."""
    if url is None:
        return False
    try:
        # Set a shorter timeout for faster validation
        response = requests.head(url, allow_redirects=True, timeout=3)
        return response.status_code == 200
    except RequestException:
        return False

# Apply the function to the 'image_url' column to create a new boolean column
df['is_url_valid'] = df['image_url'].apply(is_url_valid)

# Filter the DataFrame to keep only rows with valid URLs in the main df
df = df[df['is_url_valid']].copy()

# Display the number of valid URLs found and the first few rows of the updated DataFrame
print(f"Number of rows with valid URLs: {len(df)}")
display(df[['media', 'image_url', 'is_url_valid']].head())

Number of rows with valid URLs: 96


,media,image_url,is_url_valid
0,[Photo(previewUrl='https://pbs.twimg.com/media...,https://pbs.twimg.com/media/Eo8N3JLVoAAlDJT?fo...,True
1,[Photo(previewUrl='https://pbs.twimg.com/media...,https://pbs.twimg.com/media/Dg7mW-VX0AE_hMn?fo...,True
2,[Photo(previewUrl='https://pbs.twimg.com/media...,https://pbs.twimg.com/media/EjG2s4aXgAMNM1o?fo...,True
3,[Photo(previewUrl='https://pbs.twimg.com/media...,https://pbs.twimg.com/media/EjPaVniX0AAaWLJ?fo...,True
4,[Photo(previewUrl='https://pbs.twimg.com/media...,https://pbs.twimg.com/media/Dp4L0cSUcAAh9JG?fo...,True


In [ ]:
df.head()

,id,date,likes,content,username,media,inferred company,image_url,is_url_valid
0,1,2020-12-12 00:47:00,1,"Spend your weekend morning with a Ham, Egg, an...",TimHortonsPH,[Photo(previewUrl='https://pbs.twimg.com/media...,tim hortons,https://pbs.twimg.com/media/Eo8N3JLVoAAlDJT?fo...,True
1,2,2018-06-30 10:04:20,2750,Watch rapper <mention> freestyle for over an H...,IndyMusic,[Photo(previewUrl='https://pbs.twimg.com/media...,independent,https://pbs.twimg.com/media/Dg7mW-VX0AE_hMn?fo...,True
2,3,2020-09-29 19:47:28,57,Canadian Armenian community demands ban on mil...,CBCCanada,[Photo(previewUrl='https://pbs.twimg.com/media...,cbc,https://pbs.twimg.com/media/EjG2s4aXgAMNM1o?fo...,True
3,4,2020-10-01 11:40:09,152,"1st in Europe to be devastated by COVID-19, It...",MKWilliamsRome,[Photo(previewUrl='https://pbs.twimg.com/media...,williams,https://pbs.twimg.com/media/EjPaVniX0AAaWLJ?fo...,True
4,5,2018-10-19 14:30:46,41,Congratulations to Pauletha Butts of <mention>...,BGISD,[Photo(previewUrl='https://pbs.twimg.com/media...,independent,https://pbs.twimg.com/media/Dp4L0cSUcAAh9JG?fo...,True


In [ ]:
!pip install transformers torch Pillow requests

In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import requests

# Load the pre-trained BLIP model and processor
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

def generate_caption(image_url):
    """Generates a caption for an image from a URL using the BLIP model."""
    if image_url is None:
        return None
    try:
        # Load the image from the URL
        response = requests.get(image_url, stream=True)
        response.raise_for_status() # Raise an exception for bad status codes
        image = Image.open(response.raw).convert('RGB')

        # Prepare the image for the model
        inputs = processor(image, return_tensors="pt")

        # Generate the caption
        out = model.generate(**inputs)

        # Decode the generated caption
        caption = processor.decode(out[0], skip_special_tokens=True)
        return caption
    except Exception as e:
        print(f"Error generating caption for {image_url}: {e}")
        return None

# Apply the generate_caption function to the 'image_url' column
# This might take some time depending on the number of valid URLs
df['caption'] = df['image_url'].apply(generate_caption)

# Display the first few rows with the new 'caption' column
display(df[['image_url', 'caption']].head())

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,image_url,caption
0,https://pbs.twimg.com/media/Eo8N3JLVoAAlDJT?fo...,a sandwich and a drink on a table
1,https://pbs.twimg.com/media/Dg7mW-VX0AE_hMn?fo...,a young man in a black hoodie and headphones i...
2,https://pbs.twimg.com/media/EjG2s4aXgAMNM1o?fo...,a soldier fires a cannon during a military tra...
3,https://pbs.twimg.com/media/EjPaVniX0AAaWLJ?fo...,a beach filled with people and umbrellas
4,https://pbs.twimg.com/media/Dp4L0cSUcAAh9JG?fo...,a man and woman standing in front of a crowd


In [ ]:
API_KEY = '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'

In [ ]:
!pip install groq

In [ ]:
from groq import Groq
import os

# Assuming API_KEY is already defined in your environment or notebook
client = Groq(
    api_key=API_KEY,
)

def generate_tweet_content(row):
    """Generates tweet content using Groq API based on row attributes."""
    try:
        prompt = f"""Generate tweet content based on the following information:
        Date: {row['date']}
        Likes: {row['likes']}
        Image Caption: {row['caption']}
        Username: {row['username']}
        Company: {row['inferred company']}

        Tweet:"""

        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
            model="openai/gpt-oss-20b", # Using a suitable Groq model
        )

        return chat_completion.choices[0].message.content
    except Exception as e:
        print(f"Error generating tweet for row {row.name}: {e}")
        return None

# Apply the function to the first 5 rows of the DataFrame
# Create a new column 'generated_tweet' in the DataFrame
df['generated_tweet'] = df.head(5).apply(generate_tweet_content, axis=1)

# Display the original columns and the new generated tweet column for the first 5 rows
display(df[['date', 'likes', 'caption', 'username', 'inferred company', 'generated_tweet', 'content']].head())

,date,likes,caption,username,inferred company,generated_tweet,content
0,2020-12-12 00:47:00,1,a sandwich and a drink on a table,TimHortonsPH,tim hortons,📅 **12 Dec 2020** – 00:47 UTC \n🍔🥤 “A sandwic...,"Spend your weekend morning with a Ham, Egg, an..."
1,2018-06-30 10:04:20,2750,a young man in a black hoodie and headphones i...,IndyMusic,independent,🎶 2018‑06‑30: A young man in a black hoodie & ...,Watch rapper <mention> freestyle for over an H...
2,2020-09-29 19:47:28,57,a soldier fires a cannon during a military tra...,CBCCanada,cbc,**Tweet**\n\n> 🔫⚔️ Training day in full swing!...,Canadian Armenian community demands ban on mil...
3,2020-10-01 11:40:09,152,a beach filled with people and umbrellas,MKWilliamsRome,williams,🌊☀️ Nothing beats a sunny day on the beach! Th...,"1st in Europe to be devastated by COVID-19, It..."
4,2018-10-19 14:30:46,41,a man and woman standing in front of a crowd,BGISD,independent,**@BGISD** \n📅 2018‑10‑19 14:30:46 \n🖼️ “A m...,Congratulations to Pauletha Butts of <mention>...


In [ ]:
df['generated_tweet'][0]

'📅 **12\u202fDec\u202f2020** – 00:47\u202fUTC  \n🍔🥤 “A sandwich and a drink on a table” – just the comfort combo we all need! #TimHortons #CoffeeTime #ComfortFood\n\n👍 1  |  @TimHortonsPH |  🏪\u202fTim Hortons  '

In [ ]:
df['content'][0]

'Spend your weekend morning with a Ham, Egg, and Cheese Wrap paired with a sweet Iced French Vanilla! ☀️ Order yours now via dine-in, takeout, and delivery. #TimHortonsPH <hyperlink>'

In [ ]:
import os
import pandas as pd

# Define the folder path
folder_path = '/content/drive/MyDrive/Adobe/ParquetContent'

# Create the folder if it doesn't exist
os.makedirs(folder_path, exist_ok=True)

# Define the file path for the parquet file
file_path = os.path.join(folder_path, 'processed_tweets.parquet')

# Select all columns except 'generated_tweet'
df_to_save = df.drop(columns=['generated_tweet'])

# Save the DataFrame to a parquet file
df_to_save.to_parquet(file_path)

print(f"DataFrame saved to {file_path}")

DataFrame saved to /content/drive/MyDrive/Adobe/ParquetContent/processed_tweets.parquet


In [ ]:
# The training loss is printed during the training process.
# We can look at the output of the previous cell to find it.
# Since the training was completed, the last reported loss value will be the final training loss.

# Display the last reported training loss
print("Final Training Loss:", trainer.state.log_history[-1]['loss'])

NameError: name 'trainer' is not defined

# Task
Fine-tune a large language model using the data from the dataframe, which includes columns like 'date', 'likes', 'image caption', 'username', and 'inferred company', to predict tweet content.

## Prepare the fine-tuning dataset

### Subtask:
Format your data into the required input format for fine-tuning, which typically involves pairs of prompts and desired outputs.


**Reasoning**:
Create a list of dictionaries from the dataframe, with each dictionary containing a 'prompt' constructed from several columns and a 'completion' from the 'content' column, filtering out rows with missing values in relevant columns to prepare data for fine-tuning.



In [ ]:
finetuning_data = []

# Iterate through the DataFrame rows
for index, row in df.iterrows():
    # Check for missing values in the columns used for prompt and completion
    if pd.notna(row['date']) and pd.notna(row['likes']) and pd.notna(row['caption']) and pd.notna(row['username']) and pd.notna(row['inferred company']) and pd.notna(row['content']):
        prompt = f"Generate tweet content based on the following information:\nDate: {row['date']}\nLikes: {row['likes']}\nImage Caption: {row['caption']}\nUsername: {row['username']}\nCompany: {row['inferred company']}\nTweet:"
        completion = row['content']
        finetuning_data.append({"prompt": prompt, "completion": completion})
    else:
        print(f"Skipping row {index} due to missing values.")

# Display the first few entries of the formatted data
print(f"Number of training examples: {len(finetuning_data)}")
if len(finetuning_data) > 0:
    display(finetuning_data[:5])

**Reasoning**:
The previous command failed because the dataframe `df` was not defined in the current session. Load the data again and then create the list of dictionaries for fine-tuning.



In [ ]:
import pandas as pd

# Load the first 100 rows of the dataset
df = pd.read_csv('/content/drive/MyDrive/Adobe/TrainSet.csv', nrows=100)

# Assuming 'image_url' and 'caption' columns are already added and 'is_url_valid' filter applied
# If not, the previous steps for these would need to be included here or re-executed.
# For the purpose of continuing from the error, we'll assume df needs to be re-loaded
# and the necessary columns and filtering applied as per the previous successful steps.

# Re-apply the image URL extraction and validation (simplified, assuming the functions are defined elsewhere)
# In a real scenario, these functions (extract_image_url, is_url_valid) would need to be re-defined if not in the current session.
import re
import requests
from requests.exceptions import RequestException

def extract_image_url(media_list):
    """Extracts the preview or thumbnail URL from the media list using regex."""
    if isinstance(media_list, str):
        preview_match = re.search(r"previewUrl='(https?://[^']+)'", media_list)
        if preview_match:
            return preview_match.group(1)
        thumbnail_match = re.search(r"thumbnailUrl='(https?://[^']+)'", media_list)
        if thumbnail_match:
            return thumbnail_match.group(1)
    return None

def is_url_valid(url):
    """Checks if a URL is valid and accessible."""
    if url is None:
        return False
    try:
        response = requests.head(url, allow_redirects=True, timeout=3)
        return response.status_code == 200
    except RequestException:
        return False

df['image_url'] = df['media'].apply(extract_image_url)
df['is_url_valid'] = df['image_url'].apply(is_url_valid)
df = df[df['is_url_valid']].copy()

# Assuming the BLIP model and processor are loaded and generate_caption function is defined
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image

# Load the pre-trained BLIP model and processor (re-loading for the context)
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

def generate_caption(image_url):
    """Generates a caption for an image from a URL using the BLIP model."""
    if image_url is None:
        return None
    try:
        response = requests.get(image_url, stream=True)
        response.raise_for_status()
        image = Image.open(response.raw).convert('RGB')
        inputs = processor(image, return_tensors="pt")
        out = model.generate(**inputs)
        caption = processor.decode(out[0], skip_special_tokens=True)
        return caption
    except Exception as e:
        print(f"Error generating caption for {image_url}: {e}")
        return None

df['caption'] = df['image_url'].apply(generate_caption)


# Now, create the finetuning data list
finetuning_data = []

# Iterate through the DataFrame rows
for index, row in df.iterrows():
    # Check for missing values in the columns used for prompt and completion
    if pd.notna(row['date']) and pd.notna(row['likes']) and pd.notna(row['caption']) and pd.notna(row['username']) and pd.notna(row['inferred company']) and pd.notna(row['content']):
        prompt = f"Generate tweet content based on the following information:\nDate: {row['date']}\nLikes: {row['likes']}\nImage Caption: {row['caption']}\nUsername: {row['username']}\nCompany: {row['inferred company']}\nTweet:"
        completion = row['content']
        finetuning_data.append({"prompt": prompt, "completion": completion})
    else:
        print(f"Skipping row {index} due to missing values.")

# Display the first few entries of the formatted data
print(f"Number of training examples: {len(finetuning_data)}")
if len(finetuning_data) > 0:
    display(finetuning_data[:5])

## Choose a model

### Subtask:
Select a suitable pre-trained LLM model for fine-tuning.


**Reasoning**:
Research and identify a suitable pre-trained LLM model for fine-tuning on text generation and select its corresponding pre-trained weights from the Hugging Face Transformers library.



In [ ]:
# We will use the 'openai/gpt-oss-20b' model.
# This model is suitable for text generation tasks.

model_name = "openai/gpt-oss-20b"

print(f"Selected pre-trained model: {model_name}")

## Set up the training environment

### Subtask:
Configure the necessary libraries and settings for fine-tuning, potentially including using a platform or library that simplifies the process (e.g., Hugging Face Transformers).


**Reasoning**:
Import necessary libraries for fine-tuning, load the tokenizer, create a custom dataset class, and define training arguments as per the instructions.



In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from torch.utils.data import Dataset
import torch
import pandas as pd

# Re-create the finetuning data list
finetuning_data = []

# Iterate through the DataFrame rows
for index, row in df.iterrows():
    # Check for missing values in the columns used for prompt and completion
    if pd.notna(row['date']) and pd.notna(row['likes']) and pd.notna(row['caption']) and pd.notna(row['username']) and pd.notna(row['inferred company']) and pd.notna(row['content']):
        prompt = f"Generate tweet content based on the following information:\nDate: {row['date']}\nLikes: {row['likes']}\nImage Caption: {row['caption']}\nUsername: {row['username']}\nCompany: {row['inferred company']}\nTweet:"
        completion = row['content']
        finetuning_data.append({"prompt": prompt, "completion": completion})
    else:
        print(f"Skipping row {index} due to missing values.")

# Load the tokenizer for the chosen model and set the padding token
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Set padding token if it's not already set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


# Create a custom dataset class
class TweetDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        # Concatenate prompt and completion for language modeling
        text = item["prompt"] + item["completion"]
        # Tokenize the text
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        # For causal language modeling, the labels are the input IDs shifted
        labels = encoding["input_ids"].clone()
        # We mask the prompt part so the model only learns to predict the completion
        prompt_length = len(self.tokenizer(item["prompt"], truncation=True)["input_ids"])
        # Ensure prompt_length does not exceed the max_length after truncation
        if prompt_length > self.max_length:
            prompt_length = self.max_length
        labels[:, :prompt_length] = -100 # Use -100 to ignore loss for prompt tokens

        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
            "labels": labels.squeeze(),
        }

# Create the dataset using the finetuning_data from the previous step
train_dataset = TweetDataset(finetuning_data, tokenizer)

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./fine-tuned-gpt-oss",  # Output directory
    num_train_epochs=3,              # Number of training epochs
    per_device_train_batch_size=4,   # Batch size per device during training
    save_steps=10_000,               # Save model every x steps
    save_total_limit=2,              # Only keep the last 2 checkpoints
    logging_dir="./logs",            # Directory for storing logs
    logging_steps=200,
    learning_rate=5e-5,              # Learning rate
    weight_decay=0.01,               # Strength of weight decay
    warmup_steps=500,                # Number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    report_to="none" # Disable reporting to external services like W&B
)

print("Libraries imported, tokenizer loaded, custom dataset class created, and training arguments defined.")

## Fine-tune the model

### Subtask:
Run the fine-tuning process using your prepared dataset and chosen model.


**Reasoning**:
Load the pre-trained GPT2 model, instantiate the Trainer with the model, training arguments, and training dataset, and then start the training process.



In [ ]:
# Load the pre-trained model for language modeling
model = AutoModelForCausalLM.from_pretrained(model_name)

# Instantiate the Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

# Start the fine-tuning process
print("Starting fine-tuning process...")
trainer.train()

print("Fine-tuning process completed.")

In [ ]:
df.head()

In [ ]:
df['generated_tweet'][0]